In [ ]:
#for sentence similarity
pip install sentence-transformers

In [ ]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
import numpy as np
import nltk
import re
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
#for sentence similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset=pd.read_csv("/content/drive/MyDrive/project/duc_2001_clean.csv")
dataset = dataset.iloc[:, 1:]
dataset.head(2)

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   

                                        Stemmed_text  Words  Chars  
0  BFN 106 Exclusive report angel Paez text the s...     31    190  
1                                  Comrade Feliciano      2     17

In [ ]:
dataset = dataset.dropna()

In [ ]:
def Single_Document_Clean_Text():
  df = pd.DataFrame()
  for dir in set(dataset['Dir']):
    temp_df = dataset[dataset['Dir'] == dir]
    for doc in set(temp_df['Docno']):
      demo_df = dataset[(dataset['Dir'] == dir) & (dataset['Docno'] == doc)]
      # demo_df['Stemmed_text'] = demo_df['Stemmed_text'].apply(str)
      text = " ".join(list(demo_df['Stemmed_text']))
      dic = {'Docno': doc, 'Dir': dir, 'Text': text}
      df = df.append(dic, ignore_index = True)
  return df
single_dataset = Single_Document_Clean_Text()

In [ ]:
single_dataset.head()

Docno   Dir                                               Text
0   AP880613-0161  d53i  pol captured top military lead theShining path...
1   AP881227-0185  d53i  shin path guerrilla start theirbloody uprising...
2   AP880816-0234  d53i  a death squad opposed shin Pathguerrillas thre...
3  WSJ870501-0141  d53i  in lat 1970s poor farm living community surrou...
4        FBIS3-41  d53i  sum a lengthy statement attributed imprison sh...

In [ ]:
def Multi_Document_Clean_Text():
  df = pd.DataFrame()
  text=" "
  for dir in set(dataset['Dir']):
    temp_df = dataset[dataset['Dir'] == dir]
    for doc in set(temp_df['Docno']):
      demo_df = dataset[(dataset['Dir'] == dir) & (dataset['Docno'] == doc)]
      demo_df['Stemmed_text'] = demo_df['Stemmed_text'].apply(str)
      text = text+ " ".join(list(demo_df['Stemmed_text']))
    dic = {'Dir': dir, 'Text': text,'chars':len(text)}
    df = df.append(dic, ignore_index = True)
    text=" "
  return df
  
multi_dataset = Multi_Document_Clean_Text()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
multi_dataset.head()

Dir                                               Text    chars
0  d53i   pol captured top military lead theShining pat...  39907.0
1  d05a   CSO 106 Article Greg neal environ correspond ...  30652.0
2  d44h   GUERRILLA tact Greenpeace swamp whit House po...  30946.0
3  d41g   as last mon live pin mount Palomar are near s...  45091.0
4  d56k   Inside small mot home JoannePierluissi rais s...  14903.0

##tf-idf


In [ ]:
def get_tfidf(docno):
  temp_df = dataset[dataset['Docno'] == docno]
  a=list(temp_df['Stemmed_text'])
  tf_idf_vectorizer=TfidfVectorizer()
  tf_idf_vectorizer.fit(a)
  sentence_vectors=tf_idf_vectorizer.transform(a)
  sentence_testscore=np.array(sentence_vectors.sum(axis=1)).ravel()
  sentence_testscore=sentence_testscore.tolist()
  sentence_testscore=[float(i)/sum(sentence_testscore) for i in sentence_testscore]
  dataset.loc[dataset['Docno'] == doc,['tf-idf_score']] = sentence_testscore



In [ ]:
for doc in list(single_dataset['Docno']):
  get_tfidf(doc)

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  
0      0.017868  
1      0.004720  
2      0.014380  
3      0.022180  
4      0.017933

##sentence scoring using adverb

Adverb play an important role in summarization. It provides more information in a sentence or clause by modifying (describing) another word. They tell us more information about an action or idea.

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def get_adverb_count(text):
  wordlist=nltk.word_tokenize(text)
  adverb=[]
  for i in nltk.pos_tag(wordlist):
    if i[1] == 'RB' or i[1]=='RBR' or i[1]=='RBS':
      adverb.append(i[0])
  return len(adverb)

In [ ]:
single_dataset['adverb'] = single_dataset['Text'].apply(get_adverb_count)

In [ ]:
single_dataset.head()

Docno   Dir                                               Text  \
0   AP880613-0161  d53i  pol captured top military lead theShining path...   
1   AP881227-0185  d53i  shin path guerrilla start theirbloody uprising...   
2   AP880816-0234  d53i  a death squad opposed shin Pathguerrillas thre...   
3  WSJ870501-0141  d53i  in lat 1970s poor farm living community surrou...   
4        FBIS3-41  d53i  sum a lengthy statement attributed imprison sh...   

   adverb  
0       6  
1      20  
2       1  
3      20  
4      27

In [ ]:
def get_adverb_score(lst):
  docno = lst[1]
  text = lst[0]
  word_token = word_tokenize(text)
  adverb_list=[]
  for i in nltk.pos_tag(word_token):
    if i[1]=='RB' or i[1]=='RBR' or i[1]=='RBS':
      adverb_list.append(i[0])
  cnt = int(single_dataset.loc[single_dataset['Docno'] == docno]['adverb'])
  if cnt == 0:
    return 0
  return (len(adverb_list)/(cnt))

In [ ]:
dataset['adverb_score'] = dataset[['Stemmed_text', 'Docno']].apply(get_adverb_score, axis =1)

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  
0      0.017868       0.02439  
1      0.004720       0.00000  
2      0.014380       0.00000  
3      0.022180       0.02439  
4      0.017933       0.00000

##sentence scoring using verb

Verb play an important role in text summarization. Verbs are words that describe actions, whether physical or mental. Verbs also describe a “state of being,” like the verbs be, become, or exist.

In [ ]:
def get_verb_count(text):
  wordlist=nltk.word_tokenize(text)
  verb=[]
  for i in nltk.pos_tag(wordlist):
    if i[1] == 'VB' or i[1]=='VBG' or i[1]=='VBD':
      verb.append(i[0])
  return len(verb)

In [ ]:
single_dataset['verb'] = single_dataset['Text'].apply(get_verb_count)

In [ ]:
single_dataset.head()

Docno   Dir                                               Text  \
0   AP880613-0161  d53i  pol captured top military lead theShining path...   
1   AP881227-0185  d53i  shin path guerrilla start theirbloody uprising...   
2   AP880816-0234  d53i  a death squad opposed shin Pathguerrillas thre...   
3  WSJ870501-0141  d53i  in lat 1970s poor farm living community surrou...   
4        FBIS3-41  d53i  sum a lengthy statement attributed imprison sh...   

   adverb  verb  
0       6    47  
1      20    57  
2       1    21  
3      20    53  
4      27    45

In [ ]:
def get_verb_score(lst):
  docno = lst[1]
  text = lst[0]
  word_token = word_tokenize(text)
  verb_list=[]
  for i in nltk.pos_tag(word_token):
    if i[1]=='VB' or i[1]=='VBG' or i[1]=='VBD':
      verb_list.append(i[0])
  cnt = int(single_dataset.loc[single_dataset['Docno'] == docno]['verb'])
  if cnt == 0:
    return 0
  return (len(verb_list)/(cnt))

In [ ]:
dataset['verb_score'] = dataset[['Stemmed_text', 'Docno']].apply(get_verb_score, axis =1)

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  
0      0.017868       0.02439    0.035714  
1      0.004720       0.00000    0.000000  
2      0.014380       0.00000    0.008929  
3      0.022180       0.02439    0.026786  
4      0.017933       0.00000    0.035714

##sentence scoring using adjective

Adjective play a major role in text summarization. Adjectives are words that modify (describe) nouns. Adjectives give the reader more specific information about an object's color, size, shape, material, and more.

In [ ]:
def get_adjective_count(text):
  wordlist=nltk.word_tokenize(text)
  adj=[]
  for i in nltk.pos_tag(wordlist):
    if i[1] == 'JJ' or i[1]=='JJR' or i[1]=='JJS':
      adj.append(i[0])
  return len(adj)

In [ ]:
single_dataset['adjective'] = single_dataset['Text'].apply(get_adjective_count)

In [ ]:
single_dataset.head()

Docno   Dir                                               Text  \
0   AP880613-0161  d53i  pol captured top military lead theShining path...   
1   AP881227-0185  d53i  shin path guerrilla start theirbloody uprising...   
2   AP880816-0234  d53i  a death squad opposed shin Pathguerrillas thre...   
3  WSJ870501-0141  d53i  in lat 1970s poor farm living community surrou...   
4        FBIS3-41  d53i  sum a lengthy statement attributed imprison sh...   

   adverb  verb  adjective  
0       6    47         52  
1      20    57        118  
2       1    21          9  
3      20    53         83  
4      27    45        138

In [ ]:
def get_adjective_score(lst):
  docno = lst[1]
  text = lst[0]
  word_token = word_tokenize(text)
  adj_list=[]
  for i in nltk.pos_tag(word_token):
    if i[1]=='JJ' or i[1]=='JJR' or i[1]=='JJS':
      adj_list.append(i[0])
  cnt = int(single_dataset.loc[single_dataset['Docno'] == docno]['adjective'])
  if cnt == 0:
    return 0
  return (len(adj_list)/(cnt))

In [ ]:
dataset['adjective_score'] = dataset[['Stemmed_text', 'Docno']].apply(get_adjective_score, axis =1)

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  adjective_score  
0      0.017868       0.02439    0.035714            0.016  
1      0.004720       0.00000    0.000000            0.000  
2      0.014380       0.00000    0.008929            0.016  
3      0.022180       0.02439    0.026786            0.036  
4      0.017933       0.00000    0.035714            0.020

## text rank


TextRank uses an extractive approach and is an unsupervised graph-based text summarization technique. It is ranking of nodes(sentences) in a graph. After ranking of nodes similarly to web page ranking the scores were used for the final calculation of sentence score.

In [ ]:
def get_text_rank(docno):
  temp_df = dataset[dataset['Docno'] == docno]
  a=list(temp_df['Stemmed_text'])
  w2v=Word2Vec(a,size=1,min_count=1,iter=1000)
  sentence_embeddings=[[w2v[word][0] for word in words] for words in a]
  max_len=max([len(tokens) for tokens in a])
  sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
  similarity_matrix = np.zeros([len(a), len(a)])
  for i,row_embedding in enumerate(sentence_embeddings):
      for j,column_embedding in enumerate(sentence_embeddings):
          similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
  nx_graph = nx.from_numpy_array(similarity_matrix)
  scores = nx.pagerank(nx_graph)
  score=list(scores.values())
  dataset.loc[dataset['Docno'] == doc,['text_rank_score']] = score

In [ ]:
for doc in list(single_dataset['Docno']):
  get_text_rank(doc)

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  
0      0.017868       0.02439    0.035714            0.016         0.013294  
1      0.004720       0.00000    0.000000            0.000         0.008805  
2      0.014380       0.00000    0.008929            0.016         0.014032  
3      0.022180       0.02439    0.026786            0.036         0.011038  
4      0.017933       0.00000    0.035714            0.020         0.012965

##sentence similarity

The vocabulary overlap in between two sentences. Normalized sum of similarity to all other sentences was finally used for sentence scoring.

In [ ]:
def get_sentence_similarity(docno):
  model = SentenceTransformer('bert-base-nli-mean-tokens')
  temp_df = dataset[dataset['Docno'] == docno]
  a=list(temp_df['Stemmed_text'])
  if len(a)>1:
    sentence_embeddings = model.encode(a)
    b=cosine_similarity(
      [sentence_embeddings[0]],
      sentence_embeddings[1:]
      )
    b=b.tolist()[0]
    b.append(0)
    dataset.loc[dataset['Docno'] == doc,['sentence_similarity']] = b
  else:
    b=[0]
    dataset.loc[dataset['Docno'] == doc,['sentence_similarity']] = b
      

In [ ]:
for doc in list(single_dataset['Docno']):
  get_sentence_similarity(doc)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  \
0      0.017868       0.02439    0.035714            0.016         0.013294   
1      0.004720       0.00000    0.000000            0.000         0.008805   
2      0.014380       0.00000    0.008929            0.016         0.014032   
3      0.022180       0.02439    0.026786            0.036         0.011038   
4      0.017933       0.00000    0.035714            0.020         0.012965   

   font_style_score  sentence_position  thematic_score  sentence_similarity  
0          0.032258                1.0        0.121429             0.351307  
1          0.000000                0.8        0.007143             0.601866  
2          0.000000                0.6        0.121429             0.647553  
3          0.000000                0.4        0.221429             0.688493  
4          0.000000                0.2        0.157143             0.812094

## scoring using uppercase font

This feature gives higher weight to words written in uppercase and assign more weight to words written in title case or upper case.

In [ ]:
def get_font_count(docno):
  temp_df = dataset[dataset['Docno'] == docno]
  a=list(temp_df['Stemmed_text'].str.split(" "))
  sc=0
  for i in a[0]:
    if i.isupper():
      sc+=1
  return sc/len(a[0])
      


In [ ]:
single_dataset['font_style']=single_dataset['Docno'].apply(get_font_count)

In [ ]:
single_dataset.head(5)

Docno   Dir                                               Text  \
0   AP880613-0161  d53i  pol captured top military lead theShining path...   
1   AP881227-0185  d53i  shin path guerrilla start theirbloody uprising...   
2   AP880816-0234  d53i  a death squad opposed shin Pathguerrillas thre...   
3  WSJ870501-0141  d53i  in lat 1970s poor farm living community surrou...   
4        FBIS3-41  d53i  sum a lengthy statement attributed imprison sh...   

   adverb  verb  adjective  font_style  
0       6    47         52         0.0  
1      20    57        118         0.0  
2       1    21          9         0.0  
3      20    53         83         0.0  
4      27    45        138         0.0

In [ ]:
def get_font_count_data(lst):
  text=str(lst[0])
  docno=lst[1]
  a=list(text.split(" "))
  sc=0
  for i in a:
    if i.isupper():
      sc+=1
  return sc/len(a)

In [ ]:
dataset['font_style_score']=dataset[['Stemmed_text','Docno']].apply(get_font_count_data,axis=1)

In [ ]:
dataset.head(5)

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  \
0      0.017868       0.02439    0.035714            0.016         0.013294   
1      0.004720       0.00000    0.000000            0.000         0.008805   
2      0.014380       0.00000    0.008929            0.016         0.014032   
3      0.022180       0.02439    0.026786            0.036         0.011038   
4      0.017933       0.00000    0.035714            0.020         0.012965   

   font_style_score  
0          0.032258  
1          0.000000  
2          0.000000  
3          0.000000  
4          0.000000

##sentence position

In this sentences were scored on the basis of location of the sentence in text document. Sentence position is assumed one of most important features for sentence selection. Normally few beginning sentences and few ending sentences of document carry more important concepts than the rest of the sentences.

In [ ]:
dataset['sentence_position'] = 0.0

In [ ]:
def get_sentence_position_score(docno):
  ln=len(dataset[dataset['Docno'] == docno])
  l1=[]
  for i in range(0,ln):
    if i<=5:
      l1.append((5-i)/5)
    elif (ln-i)<5:
      l1.append((5-(ln-i))/5)
    else:
      l1.append(0)
  dataset.loc[dataset['Docno'] == doc,['sentence_position']] = l1

In [ ]:
for doc in list(single_dataset['Docno']):
  get_sentence_position_score(doc)

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  \
0      0.017868       0.02439    0.035714            0.016         0.013294   
1      0.004720       0.00000    0.000000            0.000         0.008805   
2      0.014380       0.00000    0.008929            0.016         0.014032   
3      0.022180       0.02439    0.026786            0.036         0.011038   
4      0.017933       0.00000    0.035714            0.020         0.012965   

   font_style_score  sentence_position  
0          0.032258                1.0  
1          0.000000                0.8  
2          0.000000                0.6  
3          0.000000                0.4  
4          0.000000                0.2

## thematic feature


The words that are most frequent in the document. The top n frequent words were considered as thematic words.

In [ ]:
from math import ceil
def get_thematic_keywords(text):
  w = nltk.word_tokenize(text)
  dic = nltk.FreqDist(w)
  return list(dict(dic.most_common(ceil(len(w)*0.1))))

In [ ]:
single_dataset['thematic_keyword'] = single_dataset['Text'].apply(get_thematic_keywords)
single_dataset.head()

Docno   Dir                                               Text  \
0   AP880613-0161  d53i  pol captured top military lead theShining path...   
1   AP881227-0185  d53i  shin path guerrilla start theirbloody uprising...   
2   AP880816-0234  d53i  a death squad opposed shin Pathguerrillas thre...   
3  WSJ870501-0141  d53i  in lat 1970s poor farm living community surrou...   
4        FBIS3-41  d53i  sum a lengthy statement attributed imprison sh...   

   adverb  verb  adjective  font_style  \
0       6    47         52         0.0   
1      20    57        118         0.0   
2       1    21          9         0.0   
3      20    53         83         0.0   
4      27    45        138         0.0   

                                    thematic_keyword  
0  [Morote, said, Guzman, path, shin, pol, lead, ...  
1  [path, guerrilla, the, said, shin, rebel, and,...  
2  [kil, shin, path, off, charge, said, member, t...  
3  [guerrilla, army, the, coca, shin, path, off, ...  
4  [shin, path, peace, October, cal, lead, Guzman...

In [ ]:
def get_thematic_score(lst):
  docno = lst[0]
  text = set(nltk.word_tokenize(lst[1]))
  lst = single_dataset.loc[single_dataset['Docno'] == docno, ['thematic_keyword']].iloc[0][0]
  cnt = sum([1 for word in lst if word in text])
  return (cnt/len(lst))

In [ ]:
dataset['thematic_score'] = dataset[['Docno', 'Stemmed_text']].apply(get_thematic_score, axis = 1)

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                 Removed_punctuation  \
0  BFN 106 Exclusive report by Angel Paez   Text ...   
1                                  Comrade Feliciano   
2  During the meeting the Shining Path Central Co...   
3  The document entitled To Grow Strong on the Gr...   
4  A Shining Path pamphlet dated February which i...   

                                    Removed_Stopword  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                     Lemmatized_text  \
0  BFN 106 Exclusive report Angel Paez Text The S...   
1                                  Comrade Feliciano   
2  During meeting Shining Path Central Committee ...   
3  The document entitled To Grow Strong Grounds P...   
4  A Shining Path pamphlet dated February signed ...   

                                        Stemmed_text  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  \
0      0.017868       0.02439    0.035714            0.016         0.013294   
1      0.004720       0.00000    0.000000            0.000         0.008805   
2      0.014380       0.00000    0.008929            0.016         0.014032   
3      0.022180       0.02439    0.026786            0.036         0.011038   
4      0.017933       0.00000    0.035714            0.020         0.012965   

   font_style_score  sentence_position  thematic_score  
0          0.032258                1.0        0.121429  
1          0.000000                0.8        0.007143  
2          0.000000                0.6        0.121429  
3          0.000000                0.4        0.221429  
4          0.000000                0.2        0.157143

#csv

In [ ]:
dataset=dataset.drop(['Removed_punctuation','Removed_Stopword','Lemmatized_text'],axis=1)

In [ ]:
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                      Clean_sentence  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  \
0      0.017868       0.02439    0.035714            0.016         0.013294   
1      0.004720       0.00000    0.000000            0.000         0.008805   
2      0.014380       0.00000    0.008929            0.016         0.014032   
3      0.022180       0.02439    0.026786            0.036         0.011038   
4      0.017933       0.00000    0.035714            0.020         0.012965   

   font_style_score  sentence_position  thematic_score  sentence_similarity  
0          0.032258                1.0        0.121429             0.351307  
1          0.000000                0.8        0.007143             0.601866  
2          0.000000                0.6        0.121429             0.647553  
3          0.000000                0.4        0.221429             0.688493  
4          0.000000                0.2        0.157143             0.812094

In [ ]:
dataset=dataset.rename(columns={'Stemmed_text':'Clean_sentence'})

In [ ]:
dataset.to_csv("/content/drive/MyDrive/project/Sentence_Score.csv")